In [ ]:
%pip install --upgrade langchain langchain-community langchainhub langchain-openai chromadb unstructured[all-docs] ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%pip install openai

ERROR: Operation cancelled by user


In [ ]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

False

In [ ]:
# Load, chunk and index the contents of the blog.
loader = UnstructuredFileLoader("/content/jerry_qa_pairs.txt")

docs = loader.load()

docs

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


[Document(page_content='GEORGE: Are you through? Jerry:You do of course try on, when you buy?\n\nGEORGE: How come youre not doin the second show tomorrow? Jerry:Well, theres this uh, woman might be comin in.\n\nGEORGE: Wait a second, wait a second, what coming in, what woman is coming in? Jerry:I told you about Laura, the girl I met in Michigan?\n\nGEORGE: Wait wait wait, what is she... What is she like? Jerry:Oh, shes really great. I mean, shes got like a real warmth about her and shes really bright and really pretty and uh... the conversation though, I mean, it was... talking with her is like talking with you, but, you know, obviously much better.\n\nGEORGE:  So, you know, what, what happened? Jerry:Oh, nothing happened, you know, but is was great.\n\nGEORGE:  Ho ho ho, Had to? Had to come in? Jerry:Yeah, but...\n\nGEORGE: Had to come in and maybe well get together? Had to and maybe? Jerry:Yeah!\n\nGEORGE: All right, all right, I didnt want to tell you this. You wanna know why she ca

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def create_personality_chain(retriever,personality,llm):
  instruction = f"You are a chatbot that communicates like {personality} during an interview. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just answer like {personality} would. Use three sentences maximum and keep the answer concise."
  template = instruction + '''
Context: {context}
Question: {question}
Answer: '''
  prompt = ChatPromptTemplate.from_template(template = template)
  chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
    )
  return chain

formatted_docs = format_docs(docs)
print(formatted_docs)
print(type(formatted_docs))

GEORGE: Are you through? Jerry:You do of course try on, when you buy?

GEORGE: How come youre not doin the second show tomorrow? Jerry:Well, theres this uh, woman might be comin in.

GEORGE: Wait a second, wait a second, what coming in, what woman is coming in? Jerry:I told you about Laura, the girl I met in Michigan?

GEORGE: Wait wait wait, what is she... What is she like? Jerry:Oh, shes really great. I mean, shes got like a real warmth about her and shes really bright and really pretty and uh... the conversation though, I mean, it was... talking with her is like talking with you, but, you know, obviously much better.

GEORGE:  So, you know, what, what happened? Jerry:Oh, nothing happened, you know, but is was great.

GEORGE:  Ho ho ho, Had to? Had to come in? Jerry:Yeah, but...

GEORGE: Had to come in and maybe well get together? Had to and maybe? Jerry:Yeah!

GEORGE: All right, all right, I didnt want to tell you this. You wanna know why she called you? Jerry:Yes!

GEORGE: Cause it

In [ ]:
def create_personality_chain(retriever,personality,llm):
  instruction = f"You are a chatbot that communicates like {personality} during an interview. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just answer like {personality} would. Use three sentences maximum and keep the answer concise."
  template = instruction + '''
Context: {context}
Question: {question}
Answer: '''
  prompt = ChatPromptTemplate.from_template(template = template)
  print(prompt)
  chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
    )
  return chain

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
splits = text_splitter.split_documents(docs)
print(splits)

[Document(page_content='GEORGE: Are you through? Jerry:You do of course try on, when you buy?\n\nGEORGE: How come youre not doin the second show tomorrow? Jerry:Well, theres this uh, woman might be comin in.\n\nGEORGE: Wait a second, wait a second, what coming in, what woman is coming in? Jerry:I told you about Laura, the girl I met in Michigan?\n\nGEORGE: Wait wait wait, what is she... What is she like? Jerry:Oh, shes really great. I mean, shes got like a real warmth about her and shes really bright and really pretty and uh... the conversation though, I mean, it was... talking with her is like talking with you, but, you know, obviously much better.\n\nGEORGE:  So, you know, what, what happened? Jerry:Oh, nothing happened, you know, but is was great.\n\nGEORGE:  Ho ho ho, Had to? Had to come in? Jerry:Yeah, but...\n\nGEORGE: Had to come in and maybe well get together? Had to and maybe? Jerry:Yeah!\n\nGEORGE: All right, all right, I didnt want to tell you this. You wanna know why she ca

In [ ]:
jerry_db = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
print(jerry_db)

In [ ]:
# Get all embeddings
jerry_db._collection.get(ids=['004b9577-e090-4945-b9ce-f437241d3a93'],include=['embeddings'])

{'ids': ['004b9577-e090-4945-b9ce-f437241d3a93'],
 'embeddings': [[-0.0004920387291349471,
   0.001601872849278152,
   0.02852115035057068,
   -0.0034674685448408127,
   -0.028989991173148155,
   0.005671672057360411,
   0.021801099181175232,
   -0.023298785090446472,
   -0.03490259870886803,
   -0.005740044638514519,
   0.004974922630935907,
   0.01792014017701149,
   0.006804703734815121,
   0.005730276927351952,
   -0.009559144265949726,
   0.012294049374759197,
   0.039825424551963806,
   0.001673501217737794,
   0.021019699051976204,
   -0.04427941516041756,
   0.023233668878674507,
   0.008445646613836288,
   -0.001706059672869742,
   -0.006700516678392887,
   -0.014247551560401917,
   0.02742718905210495,
   0.02729695476591587,
   -0.0035228177439421415,
   0.008061458356678486,
   -0.007039124146103859,
   -0.006824238691478968,
   0.013453126884996891,
   0.0002741009811870754,
   -0.01426057517528534,
   -0.02883371151983738,
   -0.019248520955443382,
   0.003319327952340245

In [ ]:
retriever = jerry_db.as_retriever()
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

jerry_gpt35_chain = create_personality_chain(retriever,"Jerry",llm)

# print(jerry_gpt35_chain)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are a chatbot that communicates like Jerry during an interview. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just answer like Jerry would. Use three sentences maximum and keep the answer concise.\nContext: {context}\nQuestion: {question}\nAnswer: "))]


In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

In [ ]:
embeddings = embeddings_model.embed_documents(formatted_docs[0])
len(embeddings), len(embeddings[0])
# print(embeddings)

(1, 1536)

In [ ]:
!jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

|DEBUG|Paths used for configuration of jupyter_notebook_config: 
    	/etc/jupyter/jupyter_notebook_config.json
|DEBUG|Paths used for configuration of jupyter_notebook_config: 
    	/usr/local/etc/jupyter/jupyter_notebook_config.d/panel-client-jupyter.json
    	/usr/local/etc/jupyter/jupyter_notebook_config.json
|DEBUG|Paths used for configuration of jupyter_notebook_config: 
    	/usr/etc/jupyter/jupyter_notebook_config.json
|DEBUG|Paths used for configuration of jupyter_notebook_config: 
    	/root/.local/etc/jupyter/jupyter_notebook_config.json
|DEBUG|Paths used for configuration of jupyter_notebook_config: 
    	/root/.jupyter/jupyter_notebook_config.json

  _   _          _      _
 | | | |_ __  __| |__ _| |_ ___
 | |_| | '_ \/ _` / _` |  _/ -_)
  \___/| .__/\__,_\__,_|\__\___|
       |_|
                       
Read the migration plan to Notebook 7 to learn about the new features and the actions to take if you are using extensions.

https://jupyter-notebook.readthedocs.io/en/lates